In [1]:
from dotenv import load_dotenv
from openai import OpenAI
from PyPDF2 import PdfReader
import gradio as gr
import os

In [3]:
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(api_key=openai_api_key)

In [ ]:
reader = PdfReader("me/SUKANTA BALA.pdf")
linkedin = ''
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text
print(linkedin);

